In [29]:
import pandas as pd
import numpy as np
from get_data import get_yield
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

# 1 year yield data
df_1 = get_yield(term=5)
# to weekly and take diff
df_1 = df_1.resample("W-FRI").last()
df_1 = df_1.diff().dropna()
# Skip last two months of data
df_1 = df_1.truncate(after = pd.to_datetime('2023-2-10'))


# corr = df_1.corr()

Y = df_1.copy()
Y = Y.sort_index()
Y = Y.apply(pd.to_numeric, errors="coerce")

df_1

,UK,CAN,DBR,FRN,SWISS,SEK,NOR,AUD,ITL,US,CNY
Start Date,,,,,,,,,,,
2016-02-26,-0.064,0.076,-0.043,-0.044,-0.068,-0.047,-0.026,-0.054,-0.063,0.016,-0.032
2016-03-04,0.185,0.019,0.025,0.03,0.003,0.081,-0.01,0.18,-0.079,0.134,-0.005
2016-03-11,0.123,0.117,0.065,0.014,0.117,0.025,0.035,0.12,-0.069,0.118,-0.037
2016-03-18,-0.14,-0.098,-0.016,-0.008,-0.054,-0.064,-0.101,-0.098,-0.046,-0.159,-0.041
2016-03-25,0.016,0.011,-0.012,-0.009,0.01,-0.013,-0.057,0.036,0.051,0.045,-0.035
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-13,-0.145,-0.227,-0.049,-0.042,-0.201,-0.216,-0.223,-0.26,-0.18,-0.088,0.045
2023-01-20,-0.009,-0.094,0.008,-0.022,0.078,0.083,-0.065,-0.233,0.0,-0.049,0.035
2023-01-27,-0.095,0.083,0.048,0.067,0.024,0.061,0.06,0.198,0.105,0.049,0.0


In [30]:
# Granger causlity test (just for UK-CAN for now)
# Might need to make dict of dicts for all of the combinations
gc_test_result = grangercausalitytests(Y[['CAN', 'UK']], maxlag=5, addconst=True, verbose=True)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.4424  , p=0.2305  , df_denom=360, df_num=1
ssr based chi2 test:   chi2=1.4544  , p=0.2278  , df=1
likelihood ratio test: chi2=1.4515  , p=0.2283  , df=1
parameter F test:         F=1.4424  , p=0.2305  , df_denom=360, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.2110  , p=0.1111  , df_denom=357, df_num=2
ssr based chi2 test:   chi2=4.4840  , p=0.1062  , df=2
likelihood ratio test: chi2=4.4564  , p=0.1077  , df=2
parameter F test:         F=2.2110  , p=0.1111  , df_denom=357, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4070  , p=0.2405  , df_denom=354, df_num=3
ssr based chi2 test:   chi2=4.3044  , p=0.2304  , df=3
likelihood ratio test: chi2=4.2790  , p=0.2329  , df=3
parameter F test:         F=1.4070  , p=0.2405  , df_denom=354, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.4873  , p=0.2055  

/Users/tarun/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [31]:
# VAR
model = VAR(Y)
order = model.select_order(maxlags=7)
print(order.summary())
p = order.selected_orders["aic"]
results = model.fit(p)
print(results.summary())

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -59.14     -59.02*   2.061e-26     -59.10*
1     -59.15*      -57.72  2.046e-26*      -58.58
2      -59.04      -56.29   2.300e-26      -57.94
3      -59.01      -54.95   2.371e-26      -57.39
4      -59.02      -53.64   2.372e-26      -56.88
5      -59.05      -52.36   2.336e-26      -56.39
6      -59.02      -51.01   2.459e-26      -55.83
7      -58.98      -49.66   2.624e-26      -55.27
-------------------------------------------------
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 11, Feb, 2026
Time:                     16:08:22
--------------------------------------------------------------------
No. of Equations:         11.0000    BIC:                   -57.7968
Nobs:                     363.000    HQIC:                  -58.6501
Log likelihood:   

In [32]:
# Granger Causality of everything

# Statistical meaning, things with a low enough p value improve the prediction of of Y inside the VAR, eg: lags of CAN help
# predict the UK change in yield

for caused in results.names:
    for causing in results.names:
        if caused != causing:
            test = results.test_causality(caused, [causing], kind='f')
            print(f"{causing} -> {caused}: p-value = {test.pvalue}")

CAN -> UK: p-value = 0.23406174953159017
DBR -> UK: p-value = 0.5120636443070044
FRN -> UK: p-value = 0.8149810154995946
SWISS -> UK: p-value = 0.722053866555479
SEK -> UK: p-value = 0.15246765556038416
NOR -> UK: p-value = 0.2985205127709722
AUD -> UK: p-value = 0.2824850570860747
ITL -> UK: p-value = 0.8651842846690652
US -> UK: p-value = 0.5375622869623118
CNY -> UK: p-value = 0.38277649926992796
UK -> CAN: p-value = 0.34493612311930244
DBR -> CAN: p-value = 0.7422834738069313
FRN -> CAN: p-value = 0.699306962928814
SWISS -> CAN: p-value = 0.4558940413698106
SEK -> CAN: p-value = 0.40881205883313865
NOR -> CAN: p-value = 0.1580060378894863
AUD -> CAN: p-value = 0.15559279120918276
ITL -> CAN: p-value = 0.9462667770300043
US -> CAN: p-value = 0.7724742543564189
CNY -> CAN: p-value = 0.9950007028778012
UK -> DBR: p-value = 0.24815385535960413
CAN -> DBR: p-value = 0.8782329582633126
FRN -> DBR: p-value = 0.7936312388725028
SWISS -> DBR: p-value = 0.5789774073581356
SEK -> DBR: p-value

In [33]:
fevd = results.fevd(10)
fevd.summary()

FEVD for UK
           UK       CAN       DBR       FRN     SWISS       SEK       NOR       AUD       ITL        US       CNY
0    1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000
1    0.969457  0.013378  0.000133  0.000391  0.000097  0.008896  0.002670  0.002267  0.000057  0.000684  0.001970
2    0.964859  0.016884  0.000215  0.000529  0.000101  0.009134  0.002945  0.002310  0.000065  0.000898  0.002060
3    0.964624  0.016997  0.000216  0.000529  0.000105  0.009192  0.002987  0.002310  0.000066  0.000906  0.002069
4    0.964606  0.017009  0.000216  0.000529  0.000105  0.009193  0.002989  0.002310  0.000067  0.000907  0.002069
5    0.964605  0.017009  0.000216  0.000529  0.000105  0.009193  0.002989  0.002310  0.000067  0.000907  0.002069
6    0.964605  0.017009  0.000216  0.000529  0.000105  0.009193  0.002989  0.002310  0.000067  0.000907  0.002069
7    0.964605  0.017009  0.000216  0.000529  0.000105  0.009193  0.002989  0

In [ ]:
def generalized_fevd(var_results, H=10, normalize=True):
    """
    Generalized FEVD (Pesaran-Shin) for a fitted statsmodels VARResults.
    
    Parameters
    ----------
    var_results : statsmodels.tsa.vector_ar.var_model.VARResults
        Fitted VAR results (e.g., `results = model.fit(p)`).
    H : int
        Forecast horizon in steps (e.g., weeks). Uses horizons 0..H-1 in sums.
    normalize : bool
        If True, row-normalize so contributions sum to 1 for each equation at each horizon.
        
    Returns
    -------
    fevd : np.ndarray
        Array of shape (H, n, n) where fevd[h, i, j] is contribution of shock j
        to variable i at horizon h (h=0..H-1). If normalize=True, each row sums to 1.
    names : list
        Variable names in order.
    """
    names = list(var_results.names)
    n = len(names)

    # Moving-average (MA) representation coefficients Psi_k
    # Psi has shape (H, n, n) with Psi[0] = I
    Psi = var_results.ma_rep(H)

    # Residual covariance matrix Σ (n x n)
    Sigma = np.asarray(var_results.sigma_u)

    # Precompute denominators: denom[h, i] = sum_{k=0}^{h} e_i' Psi_k Σ Psi_k' e_i
    # We'll build horizon-by-horizon contributions using cumulative sums.
    fevd = np.zeros((H, n, n), dtype=float)

    # For each horizon h, compute cumulative sums from k=0..h
    for h in range(H):
        denom = np.zeros(n, dtype=float)
        numer = np.zeros((n, n), dtype=float)

        for k in range(h + 1):
            A = Psi[k] @ Sigma  # (n x n)
            # denom_i adds (Psi_k Σ Psi_k')_ii
            denom += np.diag(A @ Psi[k].T)

            # numer_{i,j} adds (e_i' Psi_k Σ e_j)^2 / σ_jj
            # e_i' Psi_k Σ e_j is just A[i, j]
            numer += (A ** 2)

        # divide each column j by σ_jj (generalized shock scaling)
        sigma_diag = np.diag(Sigma).copy()
        # avoid division by 0 if any diag is 0 (shouldn't happen in sane VAR)
        sigma_diag[sigma_diag == 0] = np.nan

        numer = numer / sigma_diag  # broadcasts across rows

        # contribution at horizon h
        # θ_ij(h) = numer_ij / denom_i
        # (broadcast denom_i across columns)
        fevd[h] = numer / denom[:, None]

        if normalize:
            row_sums = fevd[h].sum(axis=1, keepdims=True)
            fevd[h] = fevd[h] / row_sums

    return fevd, names


def fevd_table(fevd, names, var, horizons=None):
    """
    Convenience: return a DataFrame for one dependent variable across horizons.
    var : str, dependent variable name.
    horizons : iterable of int or None -> use all.
    """
    idx = names.index(var)
    H = fevd.shape[0]
    if horizons is None:
        horizons = range(H)

    rows = []
    for h in horizons:
        rows.append(fevd[h, idx, :])

    df = pd.DataFrame(rows, columns=names, index=list(horizons))
    df.index.name = "horizon"
    return df


In [ ]:
# This creates a table which shows how much of the yields is caused locally, versus impacted from others (I think)
gfevd, names = generalized_fevd(results, H=10, normalize=True)
uk_g = fevd_table(gfevd, names, "UK")
can_g = fevd_table(gfevd, names, "CAN")
us_g = fevd_table(gfevd, names, "US")

uk_g


,UK,CAN,DBR,FRN,SWISS,SEK,NOR,AUD,ITL,US,CNY
horizon,,,,,,,,,,,
0,0.096468,0.139709,0.105810,0.094664,0.088117,0.086213,0.067226,0.085051,0.019990,0.211354,0.005400
1,0.098292,0.139136,0.105838,0.094147,0.087814,0.086355,0.068464,0.084372,0.019816,0.210378,0.005388
2,0.098323,0.139118,0.105837,0.094163,0.087802,0.086343,0.068457,0.084394,0.019823,0.210352,0.005388
3,0.098327,0.139116,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388
4,0.098326,0.139117,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388
5,0.098326,0.139117,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388
6,0.098326,0.139117,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388
7,0.098326,0.139117,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388
8,0.098326,0.139117,0.105836,0.094162,0.087805,0.086343,0.068456,0.084393,0.019824,0.210351,0.005388


In [ ]:
# This creates the Diebold-Yilmaz spillover table which is found in their papers

theta = gfevd[9]
N = theta.shape[0]

total_spillover = (theta.sum() - np.trace(theta)) / N * 100

to_others = theta.sum(axis=0) - np.diag(theta)
from_others = theta.sum(axis=1) - np.diag(theta)
net = to_others - from_others

spill = pd.DataFrame({
    "TO_others": to_others,
    "FROM_others": from_others,
    "NET": net
}, index=names).sort_values("NET", ascending=False)

total_spillover, spill

(69.98397982401123,
        TO_others  FROM_others       NET
 DBR     1.027747     0.815955  0.211792
 FRN     1.002130     0.810405  0.191726
 SWISS   0.887728     0.795611  0.092117
 US      0.878934     0.789649  0.089285
 SEK     0.827274     0.789243  0.038031
 CAN     0.810224     0.779182  0.031043
 UK      0.763231     0.765007 -0.001775
 CNY     0.016968     0.088757 -0.071788
 AUD     0.637504     0.754320 -0.116816
 NOR     0.525827     0.708310 -0.182483
 ITL     0.320668     0.601800 -0.281132)